<a href="https://colab.research.google.com/github/TapanManu/Pytorch-Repo/blob/master/Simple/AppleNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np

# Neurons

Input layer = 3

Hidden layers = 2 x 4

Output layer = 1




predict number of apples grown in a region with given temp,rainfall,humidity

In [2]:
#input(temp,rainfall,humidity)
inputs=np.array([[73,67,43],
                [91,88,64],
                [87,134,58],
                [102,43,37],
                [69,96,70]],dtype='float32')
targets = np.array([[56],
                   [81],
                   [119],
                   [22],
                   [103]],dtype='float32')

In [3]:
input=torch.from_numpy(inputs)
target=torch.from_numpy(targets)
print (input)
print (target)
#input shape
print(input.shape)
#output shape
print(target.shape)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])
torch.Size([5, 3])
torch.Size([5, 1])


In [4]:
#randomly generating weights w and biases b
#w=[[w11,w12,w13]]
#b=[b1]
#making predictive models
#apples=w11*temp+w12*rainfall+w13*humidity+b1
w=torch.randn(1,3,requires_grad=True)
b=torch.randn(1,requires_grad=True)
print(w)
print(b)

tensor([[ 0.0671,  0.6164, -0.9329]], requires_grad=True)
tensor([-2.7741], requires_grad=True)


In [5]:
from torch.utils.data import TensorDataset
#define datasets
train_ds=TensorDataset(input,target)
train_ds[0:5]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 56.],
         [ 81.],
         [119.],
         [ 22.],
         [103.]]))

In [6]:
from torch.utils.data import DataLoader
#define dataloader
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)

In [7]:
from torch import nn
import torch.nn.functional as F

In [8]:
#define the model
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.hidden1= nn.Linear(3,4)
    self.hidden2= nn.Linear(4,4)
    self.output = nn.Linear(4,1)
  
  def forward(self,x):
    x = F.relu(self.hidden1(x))
    x = F.relu(self.hidden2(x))
    x = self.output(x)
    return x

In [15]:
model = Model()
print(model)

Model(
  (hidden1): Linear(in_features=3, out_features=4, bias=True)
  (hidden2): Linear(in_features=4, out_features=4, bias=True)
  (output): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
## TODO: Specify loss and optimization functions
from torch import optim

# specify loss function
loss_fn = F.mse_loss

#define optimizer
opt=optim.Adam(model.parameters(),lr=1e-1)

In [18]:
def fit(n_epochs,model,loss_fn,opt) :
  model.train() # prep model for training

  for epoch in range(n_epochs):
      # monitor training loss
      train_loss = 0.0
      
      ###################
      # train the model #
      ###################
      for input,target in train_dl:
          # clear the gradients of all optimized variables
          opt.zero_grad()
          # forward pass: compute predicted outputs by passing inputs to the model
          output = model(input)
          # calculate the loss
          loss = loss_fn(output, target)
          # backward pass: compute gradient of the loss with respect to model parameters
          loss.backward()
          # perform a single optimization step (parameter update)
          opt.step()
          # update running training loss
          train_loss += loss.item()*input.size(0)
          #progress
          if(epoch+1)%10==0:
            print('Epoch[{}/{}],loss:{:.4f}'.format(epoch+1,n_epochs,loss.item()))
          
      # print training statistics 
      # calculate average loss over an epoch
      train_loss = train_loss/len(input)

In [19]:
fit(30000,model,loss_fn,opt)

Epoch[10/30000],loss:856.9967
Epoch[20/30000],loss:380.9665
Epoch[30/30000],loss:100.5064
Epoch[40/30000],loss:21.3962
Epoch[50/30000],loss:18.7758
Epoch[60/30000],loss:13.6592
Epoch[70/30000],loss:8.4576
Epoch[80/30000],loss:5.7755
Epoch[90/30000],loss:3.7470
Epoch[100/30000],loss:2.7894
Epoch[110/30000],loss:2.0449
Epoch[120/30000],loss:1.6087
Epoch[130/30000],loss:1.3365
Epoch[140/30000],loss:1.1742
Epoch[150/30000],loss:1.0842
Epoch[160/30000],loss:1.0347
Epoch[170/30000],loss:1.0091
Epoch[180/30000],loss:0.9964
Epoch[190/30000],loss:0.9904
Epoch[200/30000],loss:0.9876
Epoch[210/30000],loss:0.9863
Epoch[220/30000],loss:0.9857
Epoch[230/30000],loss:0.9854
Epoch[240/30000],loss:0.9851
Epoch[250/30000],loss:0.9849
Epoch[260/30000],loss:0.9847
Epoch[270/30000],loss:0.9844
Epoch[280/30000],loss:0.9842
Epoch[290/30000],loss:0.9840
Epoch[300/30000],loss:0.9838
Epoch[310/30000],loss:0.9835
Epoch[320/30000],loss:0.9833
Epoch[330/30000],loss:0.9831
Epoch[340/30000],loss:0.9828
Epoch[350/3000

In [20]:
pred = model(train_ds[:][0])
print (pred)
print (target)

tensor([[ 56.8896],
        [ 82.1926],
        [118.5479],
        [ 21.0506],
        [101.7426]], grad_fn=<AddmmBackward>)
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


In [21]:
torch.save(model.state_dict(),'samplenn.pth')

In [22]:
model.state_dict()

OrderedDict([('hidden1.weight', tensor([[-0.5535,  0.5717, -0.5425],
                      [-0.2205, -0.1074, -0.1293],
                      [-0.7473,  1.6163,  1.3241],
                      [-0.5244, -0.3466, -0.6679]])),
             ('hidden1.bias',
              tensor([ -0.3384,  -0.0277, -13.2699,  -1.3753])),
             ('hidden2.weight', tensor([[ 0.0299,  0.3897,  0.7337,  0.6353],
                      [ 0.3463, -0.0598, -0.0760,  0.3224],
                      [ 0.2168, -0.1385, -0.1628, -0.5696],
                      [ 0.3987,  0.4630, -0.7079, -0.3310]])),
             ('hidden2.bias', tensor([-2.8772, -2.1040, -0.8483, -0.5924])),
             ('output.weight', tensor([[ 0.7141,  0.1150, -0.0324,  0.5104]])),
             ('output.bias', tensor([7.9139]))])

In [24]:
model2=Model()
model2.load_state_dict(torch.load('samplenn.pth'))
model2.state_dict()

OrderedDict([('hidden1.weight', tensor([[-0.5535,  0.5717, -0.5425],
                      [-0.2205, -0.1074, -0.1293],
                      [-0.7473,  1.6163,  1.3241],
                      [-0.5244, -0.3466, -0.6679]])),
             ('hidden1.bias',
              tensor([ -0.3384,  -0.0277, -13.2699,  -1.3753])),
             ('hidden2.weight', tensor([[ 0.0299,  0.3897,  0.7337,  0.6353],
                      [ 0.3463, -0.0598, -0.0760,  0.3224],
                      [ 0.2168, -0.1385, -0.1628, -0.5696],
                      [ 0.3987,  0.4630, -0.7079, -0.3310]])),
             ('hidden2.bias', tensor([-2.8772, -2.1040, -0.8483, -0.5924])),
             ('output.weight', tensor([[ 0.7141,  0.1150, -0.0324,  0.5104]])),
             ('output.bias', tensor([7.9139]))])

In [30]:
output2 = model2(torch.FloatTensor([[73,65,49]]))
prediction = (output2)
print(prediction)

tensor([[59.3584]], grad_fn=<AddmmBackward>)
